In [1]:
pip install opencv-python numpy fpdf

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=8ba3408e6070cf2d96cead617c669a88a7039cd463b2cda55134d582a8afbfed
  Stored in directory: /root/.cache/pip/wheels/6e/62/11/dc73d78e40a218ad52e7451f30166e94491be013a7850b5d75
Successfully built fpdf


In [2]:
!pip install opencv-python moviepy ultralytics reportlab


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 52.5 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
!pip install gdown
!gdown --folder https://drive.google.com/drive/folders/13GaVu0IXX65M0WIsEH9OmkD9DlSyoXL2


Retrieving folder contents
Retrieving folder 17IUFSl14vGXZQZaQswX1wAdWpLVKKKc_ additional_ip_video
Retrieving folder 1_PF8YEyqFGqrKVPjchhdiirtIMN3sR9U ignore
Processing file 1zN-q5rheuZCFNNTkQVhKOQt4oV0xZiKe DHN-side-view-lower-2025-08-31-11-24-27-47.mp4
Processing file 1cyQukzvB9Tdfp34ORbrwJ4UEikNC99Ap lower_DHN_side_view_2.mp4
Processing file 1fpT5_GKsAkerWf-1fKmBz-UbK9xY8bgr lower_DHN_side_view1.mp4
Retrieving folder 1OyaQ-P_pK53IhDMwNmzWqJVyalzlp7BC ignore-upper-common-wagon
Processing file 1xdYEdM8Qp3Hv4CzvaxNxFtQrp5z4HuVW ip_1_Dhn-upper-side-view-2025-08-31-11-32-23-16.mp4
Processing file 1f9Dk2SAGaX8-7-qBai14WydAiAeeAnY3 1-DHN-upper-side-view-2025-08-31-11-28-15-377.mp4
Processing file 1Rzl_1mHF8AChSGnbpH9KejqbMLIYq3R_ 2-DHN-upper-side-view.mp4
Processing file 1RwqphP2oc95_wazzR8o8sByHoDXFCtX_ 3-DHN-upper-side-view.mp4
Processing file 1Isy7ZHQ711NXfDKnzetdpwXwN9uNmK64qpfph_-9j38 sample_op
Retrieving folder contents completed
Building directory structure
Building directory struct

In [5]:
import os
print(os.listdir("/content"))


['.config', 'Raw_video', 'drive', 'sample_data']


In [6]:
!pip install moviepy


In [7]:
import os

# Correct path (case-sensitive!)
video_folder = "/content/Raw_video"

# List files in the folder
print(os.listdir(video_folder))


['sample_op', '2-DHN-upper-side-view.mp4', '3-DHN-upper-side-view.mp4', 'ignore', '1-DHN-upper-side-view-2025-08-31-11-28-15-377.mp4', 'additional_ip_video', 'ignore-upper-common-wagon']


In [8]:
import os
print(os.listdir("/content"))



['.config', 'Raw_video', 'drive', 'sample_data']


In [9]:
import os

# Correct path (case-sensitive!)
video_folder = "/content/Raw_video"

# List files in the folder
print(os.listdir(video_folder))


['sample_op', '2-DHN-upper-side-view.mp4', '3-DHN-upper-side-view.mp4', 'ignore', '1-DHN-upper-side-view-2025-08-31-11-28-15-377.mp4', 'additional_ip_video', 'ignore-upper-common-wagon']


In [10]:
# Paste entire block into a Colab code cell and run
import os, cv2, math, glob
import numpy as np
from pathlib import Path
from moviepy.editor import VideoFileClip
from scipy.ndimage import gaussian_filter1d
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
from datetime import datetime

# ---------- config ----------
RAW_DIR = "/content/Raw_video"           # your downloaded folder (case-sensitive)
OUTPUT_ROOT = "/content/Processed_Video" # local outputs
DRIVE_SAVE = "/content/drive/MyDrive/Processed_Video"  # optional: save to Drive if mounted
TRAIN_ID_PREFIX = "DHN"  # used in naming if videos don't have train numbers
N_MAX_THUMBNAILS = 6
FRAME_SNAPSHOT_EVERY = 30  # save a frame every N frames when creating frame pool
MIN_COACH_WIDTH_PX = 40
SMOOTH_SIGMA = 12
VALLEY_PROM = 0.22

# Create outputs
os.makedirs(OUTPUT_ROOT, exist_ok=True)

# ---------- helper funcs ----------
def ensure_dir(p): Path(p).mkdir(parents=True, exist_ok=True)

def aggregate_foreground_mask(video_path, max_frames=None, skip=1):
    cap = cv2.VideoCapture(str(video_path))
    fgbg = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=24, detectShadows=False)
    agg = None
    frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret: break
        if frame_idx % skip != 0:
            frame_idx += 1
            continue
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        fg = fgbg.apply(gray)
        _, fg = cv2.threshold(fg, 200, 255, cv2.THRESH_BINARY)
        # morphological clean
        fg = cv2.morphologyEx(fg, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))
        fg = cv2.morphologyEx(fg, cv2.MORPH_CLOSE, np.ones((7,7), np.uint8))
        if agg is None:
            agg = fg
        else:
            agg = cv2.bitwise_or(agg, fg)
        frame_idx += 1
        if max_frames and frame_idx >= max_frames:
            break
    cap.release()
    return agg

def bounding_box_from_mask(mask):
    ys, xs = np.where(mask > 0)
    if len(xs)==0:
        return None
    x1, x2 = int(xs.min()), int(xs.max())
    y1, y2 = int(ys.min()), int(ys.max())
    return (x1, y1, x2, y2)

def find_coach_splits(agg_mask_crop, min_coach_width=MIN_COACH_WIDTH_PX, smooth_sigma=SMOOTH_SIGMA, valley_prominence=VALLEY_PROM):
    proj = np.sum(agg_mask_crop > 0, axis=0).astype(np.float32)
    if proj.max() == 0:
        return [(0, agg_mask_crop.shape[1]-1)]
    proj_sm = gaussian_filter1d(proj, sigma=smooth_sigma)
    proj_sm = proj_sm / (proj_sm.max()+1e-9)
    thresh = valley_prominence * proj_sm.max()
    low_mask = proj_sm < thresh
    intervals = []
    N = len(proj_sm)
    i = 0
    while i < N:
        if low_mask[i]:
            j = i
            while j < N and low_mask[j]:
                j += 1
            intervals.append((i, j-1))
            i = j
        else:
            i += 1
    split_positions = [(a+b)//2 for (a,b) in intervals if (b-a)>=1]
    if not split_positions:
        return [(0, N-1)]
    bounds = []
    left = 0
    for s in split_positions:
        if (s - left) >= min_coach_width:
            bounds.append((left, s))
            left = s+1
    if (N-1 - left) >= min_coach_width:
        bounds.append((left, N-1))
    if not bounds:
        return [(0, N-1)]
    return bounds

def write_per_coach_videos(src_video_path, train_bbox, coach_x_intervals, out_root, train_number):
    cap = cv2.VideoCapture(str(src_video_path))
    fps = cap.get(cv2.CAP_PROP_FPS) or 25.0
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    x1,y1,x2,y2 = train_bbox
    w_total = x2 - x1 + 1
    h_total = y2 - y1 + 1
    writers = []
    coach_folders = []
    for i,(xs, xe) in enumerate(coach_x_intervals, start=1):
        folder = Path(out_root) / f"{train_number}_{i}"
        ensure_dir(folder)
        ensure_dir(folder/"frames")
        ensure_dir(folder/"annotated")
        out_path = folder / f"{train_number}_{i}.mp4"
        w = int(xe - xs + 1)
        h = int(h_total)
        writer = cv2.VideoWriter(str(out_path), fourcc, fps, (w, h))
        writers.append((writer, xs))
        coach_folders.append(folder)
    idx = 0
    saved_frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret: break
        idx += 1
        crop = frame[y1:y2+1, x1:x2+1]
        # write full crop subvideos
        for j, (vw, xs) in enumerate(writers):
            rel_xs = xs - x1
            rel_xe = rel_xs + int(vw.get(cv2.CAP_PROP_FRAME_WIDTH) or (crop.shape[1]-rel_xs)) - 1
            rel_xs = max(0, rel_xs)
            rel_xe = min(crop.shape[1]-1, rel_xe)
            if rel_xs >= rel_xe: continue
            sub = crop[:, rel_xs:rel_xe+1]
            # ensure same shape as writer expects
            hexp, wexp = sub.shape[:2]
            # if mismatch, resize to writer (safer)
            writer_size = (int(vw.get(cv2.CAP_PROP_FRAME_WIDTH)) or sub.shape[1],
                           int(vw.get(cv2.CAP_PROP_FRAME_HEIGHT)) or sub.shape[0])
            if (wexp, hexp) != writer_size:
                sub = cv2.resize(sub, writer_size)
            vw.write(sub)
            # snapshot frame pool for selection every N frames
            if idx % FRAME_SNAPSHOT_EVERY == 0:
                frame_fname = coach_folders[j] / "frames" / f"{train_number}_{j+1}_{idx:06d}.jpg"
                cv2.imwrite(str(frame_fname), sub)
                saved_frame_count += 1
    cap.release()
    for vw,_ in writers:
        vw.release()
    return coach_folders

def select_minimal_frames_for_coverage(frames_folder, coach_width_px, n_max=N_MAX_THUMBNAILS):
    files = sorted(list(Path(frames_folder).glob("*.jpg")))
    if not files:
        return []
    coverages = []
    for f in files:
        img = cv2.imread(str(f), cv2.IMREAD_GRAYSCALE)
        if img is None: continue
        col_sum = np.sum(img>10, axis=0)
        covered_cols = np.where(col_sum>0)[0]
        if covered_cols.size==0:
            coverage = (0, coach_width_px-1)
        else:
            coverage = (int(covered_cols[0]), int(covered_cols[-1]))
        coverages.append((f, coverage))
    uncovered = set(range(coach_width_px))
    selected = []
    while uncovered and coverages and len(selected) < n_max:
        best = None; best_new = -1; best_cov = None
        for f,cov in coverages:
            a,b = cov
            new = len(uncovered & set(range(a,b+1)))
            if new > best_new:
                best_new = new; best = f; best_cov = cov
        if best is None or best_new==0:
            break
        selected.append((best, best_cov))
        a,b = best_cov
        uncovered -= set(range(a, b+1))
        coverages = [(f,c) for (f,c) in coverages if f!=best]
    # fallback: if none selected choose up to n_max earliest frames
    if not selected:
        for f in files[:n_max]:
            selected.append((f, (0, coach_width_px-1)))
    return selected

def make_pdf_report(pdf_path, train_name, coaches_info):
    c = canvas.Canvas(str(pdf_path), pagesize=A4)
    w, h = A4
    c.setFont("Helvetica-Bold", 16)
    c.drawString(40, h-40, f"Train Report: {train_name}")
    c.setFont("Helvetica", 12)
    c.drawString(40, h-70, f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    c.drawString(40, h-100, f"Total Coaches (detected): {len(coaches_info)}")
    y = h-140
    for i,info in enumerate(coaches_info, start=1):
        c.drawString(40, y, f"Wagon{i}: selected frames = {len(info['selected_frames'])}")
        y -= 18
        if y < 80:
            c.showPage()
            y = h-80
    # per-coach pages with thumbnails
    for i, info in enumerate(coaches_info, start=1):
        c.showPage()
        c.setFont("Helvetica-Bold", 14)
        c.drawString(40, h-40, f"Wagon {i} - {info.get('folder_name','')}")
        y = h-80
        x = 40
        thumb_w = 260
        thumb_h = 140
        for (f, cov) in info['selected_frames'][:6]:
            try:
                c.drawImage(str(f), x, y-thumb_h, width=thumb_w, height=thumb_h, preserveAspectRatio=True)
            except Exception as e:
                pass
            y -= (thumb_h + 20)
            if y < 120:
                c.showPage()
                y = h-80
    c.save()

# ---------- main processing ----------
def process_video_file(video_path, out_root=OUTPUT_ROOT, train_number=None):
    print("Processing:", video_path)
    if train_number is None:
        train_number = TRAIN_ID_PREFIX + "_" + Path(video_path).stem
    # aggregate FG
    agg = aggregate_foreground_mask(video_path, skip=2, max_frames=None)
    if agg is None:
        print("No foreground detected; skipping")
        return
    bbox = bounding_box_from_mask(agg)
    if bbox is None:
        print("Could not find train bbox; skipping")
        return
    x1,y1,x2,y2 = bbox
    # crop the aggregated to the bbox and find splits
    agg_crop = agg[y1:y2+1, x1:x2+1]
    coach_bounds = find_coach_splits(agg_crop)
    abs_intervals = [(x1 + a, x1 + b) for (a,b) in coach_bounds]
    print(f"Detected {len(abs_intervals)} coaches (intervals): {abs_intervals}")
    # write per-coach videos & frames
    video_out_root = Path(out_root)/ Path(video_path).stem
    ensure_dir(video_out_root)
    coach_folders = write_per_coach_videos(video_path, bbox, abs_intervals, str(video_out_root), train_number)
    coaches_info = []
    for i, folder in enumerate(coach_folders, start=1):
        frames_folder = folder/"frames"
        width_px = abs_intervals[i-1][1] - abs_intervals[i-1][0] + 1
        selected = select_minimal_frames_for_coverage(frames_folder, width_px, n_max=N_MAX_THUMBNAILS)
        coaches_info.append({
            "folder_name": folder.name,
            "selected_frames": selected
        })
    # generate PDF report
    pdf_path = video_out_root / f"Report_{train_number}.pdf"
    make_pdf_report(pdf_path, train_number, coaches_info)
    print("Saved outputs to:", video_out_root)
    return video_out_root

# Run on all mp4s in RAW_DIR
videos = [os.path.join(RAW_DIR, f) for f in os.listdir(RAW_DIR) if f.lower().endswith(".mp4")]
print("Found videos:", videos)
all_outputs = []
for v in videos:
    out = process_video_file(v)
    all_outputs.append(out)

# Optional: copy outputs to Drive if mounted
if os.path.exists("/content/drive"):
    for out in all_outputs:
        if out is None: continue
        dst = os.path.join(DRIVE_SAVE, out.name)
        ensure_dir(dst)
        # copy tree (simple)
        os.system(f"cp -r {str(out)} {dst}")
    print("Copied outputs to Drive folder:", DRIVE_SAVE)


/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:294: SyntaxWarning: invalid escape sequence '\d'
  lines_video = [l for l in lines if ' Video: ' in l and re.search('\d+x\d+', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:367: SyntaxWarning: invalid escape sequence '\d'
  rotation_lines = [l for l in lines if 'rotate          :' in l and re.search('\d+$', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:370: SyntaxWarning: invalid escape sequence '\d'
  match = re.search('\d+$', rotation_line)
  if event.key is 'enter':



Found videos: ['/content/Raw_video/2-DHN-upper-side-view.mp4', '/content/Raw_video/3-DHN-upper-side-view.mp4', '/content/Raw_video/1-DHN-upper-side-view-2025-08-31-11-28-15-377.mp4']
Processing: /content/Raw_video/2-DHN-upper-side-view.mp4
Detected 1 coaches (intervals): [(0, 2399)]
Saved outputs to: /content/Processed_Video/2-DHN-upper-side-view
Processing: /content/Raw_video/3-DHN-upper-side-view.mp4
Detected 1 coaches (intervals): [(0, 2399)]
Saved outputs to: /content/Processed_Video/3-DHN-upper-side-view
Processing: /content/Raw_video/1-DHN-upper-side-view-2025-08-31-11-28-15-377.mp4
Detected 1 coaches (intervals): [(0, 1919)]
Saved outputs to: /content/Processed_Video/1-DHN-upper-side-view-2025-08-31-11-28-15-377
Copied outputs to Drive folder: /content/drive/MyDrive/Processed_Video


In [11]:
# Optional YOLO inference on selected thumbnails
from ultralytics import YOLO
model = YOLO("yolov8n.pt")  # small pretrained; change to your custom .pt if available

# loop through processed outputs and annotate selected frames
for out in all_outputs:
    if out is None: continue
    # find report and coaches_info by reading the frames directories
    coach_dirs = sorted([d for d in out.iterdir() if d.is_dir()])
    for cd in coach_dirs:
        sel_files = sorted((cd/"frames").glob("*.jpg"))
        sel_files = sel_files[:N_MAX_THUMBNAILS]  # top ones to run
        for img_path in sel_files:
            res = model.predict(source=str(img_path), imgsz=640, conf=0.25, save=False)
            # use res[0].boxes to draw boxes
            img = cv2.imread(str(img_path))
            if img is None: continue
            boxes = res[0].boxes
            for box in boxes:
                x1,y1,x2,y2 = map(int, box.xyxy[0].tolist())
                cls = int(box.cls[0].item())
                conf = float(box.conf[0].item())
                name = res[0].names.get(cls, str(cls))
                cv2.rectangle(img, (x1,y1), (x2,y2), (0,255,0), 2)
                cv2.putText(img, f"{name} {conf:.2f}", (x1, max(0,y1-8)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 1)
            out_annot = cd/"annotated"/f"annot_{Path(img_path).name}"
            cv2.imwrite(str(out_annot), img)
print("YOLO annotation pass complete (if run).")


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.

image 1/1 /content/Processed_Video/2-DHN-upper-side-view/DHN_2-DHN-upper-side-view_1/frames/DHN_2-DHN-upper-side-view_1_000030.jpg: 288x640 1 train, 355.9ms
Speed: 10.4ms preprocess, 355.9ms inference, 37.5ms postprocess per image at shape (1, 3, 288, 640)

image 1/1 /content/Processed_Video/2-DHN-upper-side-view/DHN_2-DHN-upper-side-view_1/frames/DHN_2-DHN-upper-side-view_1_000060.jpg: 288x640 1 train, 152.1ms
Speed: 3.3ms preprocess, 152.1ms inference, 1.6ms postprocess per image at shape (1, 3, 288, 640)

image 1/1 /content/Processed_Video/2-DHN-upper-side-view/DHN_2-DHN-upper-side-view_1/frames/DHN_2-DHN-upper-side-view_1_000090.jpg: 288x640 1 airplane, 1 train, 145.1ms
Speed

In [14]:
import shutil

shutil.make_archive("/content/Processed_Output", 'zip', "/content/Processed_Video")


'/content/Processed_Output.zip'

In [15]:
from google.colab import files
files.download("/content/Processed_Output.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>